In [1]:
import pandas as pd
import sqlite3
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
pd.set_option("display.max_columns",75)


In [2]:
conn = sqlite3.Connection("../artifacts/data/db.sqlite3")

dat = pd.read_sql('select * from FEATURES',
con=conn)

In [3]:
features = ['adjoe', 'adjde', 'barthag',
       'efg_pct', 'efgd_pct', 'tor', 'tord', 'orb', 'drb', 'ftr', 'ftrd',
       '2p_pct', '2pd_pct', '3p_pct', '3pd_pct',
       'win_perc', 
       'WCC', 'Amer', 'B12', 'ACC', 'SEC',
       'BE', 'P12', 'B10', 'MWC', 'MVC', 'A10', 'OVC', 'CUSA', 'AE', 'SC',
       'WAC', 'Sum', 'CAA', 'MAAC', 'MAC', 'Ivy', 'ASun', 'Pat', 'SB', 'BW',
       'BSth', 'BSky', 'NEC', 'Horz', 'SWAC', 'MEAC', 'Slnd'
       ]
target = 'OUTCOME'

In [4]:
training_mask = dat['year'].isin([2008,
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2021,])
validation_mask = dat['year'].isin([2022])
X = dat[training_mask][features].copy()
y = dat[training_mask][target].copy()

In [5]:
model = Pipeline(
    steps=[
    ('scaler',MinMaxScaler()),
    ('learner',RandomForestRegressor(n_estimators=500,random_state=50,#max_depth=8
                                     ))
    ]
)

In [6]:
model.fit(X,y)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('learner',
                 RandomForestRegressor(n_estimators=500, random_state=50))])

In [7]:
yfit = model.predict(X)

In [8]:
Xx = dat[validation_mask][features].copy()
yy = dat[validation_mask][target].copy()
yval = model.predict(Xx)

In [9]:
val_result = pd.DataFrame({
    'team':dat[validation_mask]['team'],
    'OUTCOME':yy})
val_result['PREDICTION_NUMERIC'] = yval
val_result['PREDICTION_RANK']=val_result['PREDICTION_NUMERIC'].rank(ascending=True,)

In [10]:
val_result.sort_values(by='PREDICTION_RANK',ascending=True).head(50)

,team,OUTCOME,PREDICTION_NUMERIC,PREDICTION_RANK
4527,Kansas,1,10.674,1.0
4524,Gonzaga,16,10.864,2.0
4525,Houston,8,11.408,3.0
4526,Baylor,32,14.876,4.0
4531,Kentucky,64,17.504,5.0
4528,Arizona,16,18.580,6.0
4529,Duke,4,18.692,7.0
4537,Iowa,64,19.974,8.0
4533,Villanova,4,21.376,9.0
4536,UCLA,16,21.802,10.0
